<a href="https://colab.research.google.com/github/praffuln/generative-ai/blob/master/rag/rag-1/rag_routing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('hello world')

hello world


**Enviornment**

In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain youtube-transcript-api pytube google-generativeai langchain_google_genai langchain_community


# Setup Keys For GOOGLE_API_KEY AND LANGSMITH

In [40]:
# Assigning value to variable
GOOGLE_API_KEY=''
LANGCHAIN_API_KEY = ''


# Imports os, google Gemini EMbedding and Chat Classes with FAISS vectorstores

In [8]:
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS



# setup langsmith for tracing

In [9]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

## [Logical and Semantic routing]

Use function-calling for classification.



In [41]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

# LLM with function call
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3) # Try using gemini-pro-vision
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source by selecting one of the following options:

python_docs
js_docs
golang_docs
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router
router = prompt | structured_llm
print(router)


first=ChatPromptTemplate(input_variables=['question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an expert at routing a user question to the appropriate data source.\n\nBased on the programming language the question is referring to, route it to the relevant data source by selecting one of the following options:\n\npython_docs\njs_docs\ngolang_docs \n')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]) middle=[RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', temperature=0.3, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x77ff351f22c0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x77ff4448a440>, default_metadata=()), kwargs={'tools': [{'function_declarations': [{'name': 'RouteQuery', 'description

In [35]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})
print(result)


datasource='python_docs'


In [36]:
result

RouteQuery(datasource='python_docs')

In [37]:
result.datasource


'python_docs'

In [38]:
def choose_route(result):
    if "python_docs" in result.datasource.lower():
        ### Logic here
        return "chain for python_docs"
    elif "js_docs" in result.datasource.lower():
        ### Logic here
        return "chain for js_docs"
    else:
        ### Logic here
        return "golang_docs"

from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route)


In [39]:
full_chain.invoke({"question": question})


'chain for python_docs'

# Semantic routing


In [44]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Two prompts
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

# Embed prompts
# Create embeddings using a Google Generative AI model
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Route question to prompt
def prompt_router(input):
    # Embed question
    query_embedding = embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)

print(chain.invoke("What's a black hole"))

print(chain.invoke("What's multiplication of 4 and 2"))


Using PHYSICS
A black hole is a region of spacetime where gravity is so strong that nothing, not even light, can escape. Imagine a star much bigger than our sun, collapsing under its own gravity. As it collapses, it becomes incredibly dense and its gravitational pull intensifies.  

Think of it like a giant vacuum cleaner in space.  Anything that gets too close gets sucked in, never to be seen again. 

We can't actually see black holes directly because they don't emit light. But we can detect their presence by observing their effects on nearby stars and gas. 

It's a fascinating and complex topic, and there's still much we don't know about black holes. But hopefully, this gives you a basic understanding! 

Using MATH
Multiplication of 4 and 2 is simply **8**. 

Here's how we break it down:

* **Component 1:**  Understanding multiplication. Multiplication is repeated addition.
* **Component 2:**  Applying the concept.  4 multiplied by 2 means adding 4 to itself 2 times: 4 + 4 = 8.

Ther